In [2]:
import torch
import matplotlib.pyplot as plt 
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
with open("./names.txt") as f:
    words = f.read().splitlines()
    print(len(words), max(len(w) for w in words), min(len(w) for w in words))

chars = sorted(list(set((''.join(words)))))  #get unique characters 
stoi = {s:i+1 for i, s in enumerate(chars)}  # map char to int
stoi["."] = 0 
itos = {i:s for s,i in stoi.items()}         # map int to char
vocab_size = len(itos)


# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


32033 15 2
torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [4]:
n_dim = 8 
n_hidden = 164
class WaveNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(vocab_size,n_dim)
        self.seq = nn.Sequential(
            
            nn.BatchNorm1d(n_hidden),
            nn.Linear(n_hidden, vocab_size),
        nn.BatchNorm1d(vocab_size)
        )
    def forward(self, x):
        emb = self.emb(x).view(-1, n_dim*block_size)
        return self.seq(emb)

In [5]:
class FlattenConsecutive:
  
  def __init__(self, n):
    self.n = n
    
  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T//self.n, C*self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out
  
  def parameters(self):
    return []


In [6]:
m = nn.Conv1d(16, 33, 3, stride=4)
input = torch.randn(20, 16, 50)
output = m(input)

In [7]:
output.shape

torch.Size([20, 33, 12])

In [8]:
16*0.75

12.0

In [10]:
a = torch.randn(1, 16, 5)

In [11]:
nn.Conv1d(a)

TypeError: __init__() missing 2 required positional arguments: 'out_channels' and 'kernel_size'

In [13]:
m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
output.shape

torch.Size([20, 33, 24])

In [ ]:
a = torch.ones(10, 4,5)
